In [1]:
import pandas as pd
import numpy as np
import pywt
from scipy.signal import savgol_filter, medfilt
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skimage.restoration import denoise_wavelet
from cv2 import bilateralFilter
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Load dataset
data = pd.read_csv('PrinterSimulatorDataset.csv', low_memory=False)
data.columns = data.columns.str.strip()
print(data.head())


# 👣 FEATURE ENGINEERING
def map_size(x):
    if x == 220:
        return 'Small'
    elif x == 330:
        return 'Medium'
    elif x == 560:
        return 'Large'
    else:
        return 'Unknown'

if 'Size X' in data.columns:
    data['Size'] = data['Size X'].apply(map_size)
    data.drop(columns=['Size X', 'Size Y', 'AR', 'ASR','PadID'], inplace=True, errors='ignore')
else:
    print("Warning: 'Size X' not found.")
    data['Size'] = 'Unknown'

size_map = {'Small': 0, 'Medium': 1, 'Large': 2, 'Unknown': -1}
data['Size'] = data['Size'].map(size_map)

# Encode categorical features
categorical_cols = ['Cleaning Type', 'Direction', 'Size']
le = LabelEncoder()
for col in categorical_cols:
    if col in data.columns:
        data[col] = le.fit_transform(data[col].astype(str))

print(data.head())

# Define features and targets
targets = ['Volume', 'OffsetX', 'OffsetY']
features = list(data.columns.difference(targets))

X = data[features]
y = data[targets]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


   PadID  Size X  Size Y  Volume  OffsetX  OffsetY  PCB ID  Printing Speed  \
0      1     220     220  74.447   -9.990    5.766     241              30   
1      2     220     220  84.224  -15.069   17.116     241              30   
2      3     220     220  77.646   -5.895    5.528     241              30   
3      4     220     220  79.544  -11.337    8.486     241              30   
4      5     220     220  79.548  -13.730    8.963     241              30   

   Printing Pressure  Separation Speed Cleaning Type  Cleaning Age Direction  \
0                 60               3.0           Wet             1         F   
1                 60               3.0           Wet             1         F   
2                 60               3.0           Wet             1         F   
3                 60               3.0           Wet             1         F   
4                 60               3.0           Wet             1         F   

   Pos X    Pos Y  Rotation      AR   ASR  
0  312

***Decision Tree Regressor***

In [5]:
# Filtering functions
def apply_dtcwt_filter(signal):
    coeffs = pywt.wavedec(signal, 'db4', level=1)
    coeffs[1:] = [np.zeros_like(i) for i in coeffs[1:]]
    return pywt.waverec(coeffs, 'db4')[:len(signal)]

def apply_savgol_filter(signal):
    return savgol_filter(signal, window_length=min(len(signal), 21), polyorder=2)

def apply_bilateral_filter(signal):
    return bilateralFilter(signal.astype(np.float32), 9, 75, 75)

def apply_median_filter(signal):
    return medfilt(signal, kernel_size=3)

def apply_multivariate_filter(signal):
    return np.convolve(signal, np.ones(5)/5, mode='same')

def apply_gaussian_filter(signal):
    return gaussian_filter(signal, sigma=1)
# Filtering options
filters = {
    "DTCWT + Savitzky-Golay": lambda x: apply_savgol_filter(apply_dtcwt_filter(x)),
    "DTCWT + Bilateral": lambda x: apply_bilateral_filter(apply_dtcwt_filter(x)),
    "DTCWT + Median": lambda x: apply_median_filter(apply_dtcwt_filter(x)),
    "DTCWT + Multivariate": lambda x: apply_multivariate_filter(apply_dtcwt_filter(x)),
    "DTCWT + Gaussian": lambda x: apply_gaussian_filter(apply_dtcwt_filter(x)),
}

# Prepare results
results = []

for filter_name, filter_func in filters.items():
    filtered_data = y.copy()
    for col in targets:
        filtered_data[col] = filter_func(y[col])

    # Standardization
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    # Train Decision Tree Regressor
    for target in targets:
        dt_model = DecisionTreeRegressor(random_state=42)
        dt_model.fit(X_train_scaled, filtered_data.loc[y_train.index, target])

        y_pred_dt = dt_model.predict(X_test_scaled)
        rmse_dt = np.sqrt(mean_squared_error(y_test[target], y_pred_dt))
        mae_dt = mean_absolute_error(y_test[target], y_pred_dt)

        results.append([filter_name, target, rmse_dt, mae_dt])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Filter + Model', 'Target Variable', 'RMSE', 'MAE'])

# Print results in a structured way
print(results_df.pivot(index='Filter + Model', columns='Target Variable', values=['RMSE', 'MAE']))


                            RMSE                           MAE            \
Target Variable          OffsetX   OffsetY    Volume   OffsetX   OffsetY   
Filter + Model                                                             
DTCWT + Bilateral       3.531772  3.859028  3.667762  2.729022  2.948271   
DTCWT + Gaussian        3.777170  4.099107  3.818528  2.918604  3.139613   
DTCWT + Median          4.276956  4.623019  4.225402  3.313960  3.546011   
DTCWT + Multivariate    3.469284  3.786472  3.590935  2.678676  2.892800   
DTCWT + Savitzky-Golay  3.730837  4.090557  3.897161  2.887450  3.123546   

                                  
Target Variable           Volume  
Filter + Model                    
DTCWT + Bilateral       2.776084  
DTCWT + Gaussian        2.874029  
DTCWT + Median          3.174201  
DTCWT + Multivariate    2.712480  
DTCWT + Savitzky-Golay  2.966612  


***SVR***

In [7]:
from sklearn.svm import LinearSVR

# Filtering functions
def apply_dtcwt_filter(signal):
    coeffs = pywt.wavedec(signal, 'db4', level=1)
    coeffs[1:] = [np.zeros_like(i) for i in coeffs[1:]]
    return pywt.waverec(coeffs, 'db4')[:len(signal)]

def apply_savgol_filter(signal):
    return savgol_filter(signal, window_length=min(len(signal), 21), polyorder=2)

def apply_bilateral_filter(signal):
    return bilateralFilter(signal.astype(np.float32), 9, 75, 75)

def apply_median_filter(signal):
    return medfilt(signal, kernel_size=3)

def apply_multivariate_filter(signal):
    return np.convolve(signal, np.ones(5)/5, mode='same')

def apply_gaussian_filter(signal):
    return gaussian_filter(signal, sigma=1)

# Filtering options
filters = {
    "DTCWT + Savitzky-Golay": lambda x: apply_savgol_filter(apply_dtcwt_filter(x)),
    "DTCWT + Bilateral": lambda x: apply_bilateral_filter(apply_dtcwt_filter(x)),
    "DTCWT + Median": lambda x: apply_median_filter(apply_dtcwt_filter(x)),
    "DTCWT + Multivariate": lambda x: apply_multivariate_filter(apply_dtcwt_filter(x)),
    "DTCWT + Gaussian": lambda x: apply_gaussian_filter(apply_dtcwt_filter(x)),
}

# Prepare results
results = []

for filter_name, filter_func in filters.items():
    filtered_data = y.copy()
    for col in targets:
        filtered_data[col] = filter_func(y[col])

    # Standardization
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(filtered_data.loc[y_train.index])
    y_test_scaled = scaler_y.transform(filtered_data.loc[y_test.index])

    # Train SVR
    for i, target in enumerate(targets):
        svr_model = LinearSVR(max_iter=1000, random_state=42)
        svr_model.fit(X_train_scaled, y_train_scaled[:, i])

        y_pred_svr = svr_model.predict(X_test_scaled)
        rmse_svr = np.sqrt(mean_squared_error(y_test_scaled[:, i], y_pred_svr))
        mae_svr = mean_absolute_error(y_test_scaled[:, i], y_pred_svr)

        results.append([filter_name, target, rmse_svr, mae_svr])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Filter + Model', 'Target Variable', 'RMSE', 'MAE'])

# Print results in a structured way
print(results_df.pivot(index='Filter + Model', columns='Target Variable', values=['RMSE', 'MAE']))


                            RMSE                           MAE            \
Target Variable          OffsetX   OffsetY    Volume   OffsetX   OffsetY   
Filter + Model                                                             
DTCWT + Bilateral       0.804924  0.385820  0.840030  0.645018  0.299732   
DTCWT + Gaussian        0.809186  0.391201  0.847613  0.648167  0.303763   
DTCWT + Median          0.815022  0.396292  0.853232  0.652104  0.307988   
DTCWT + Multivariate    0.806419  0.387887  0.843932  0.645995  0.301342   
DTCWT + Savitzky-Golay  0.804519  0.385782  0.837915  0.644999  0.299464   

                                  
Target Variable           Volume  
Filter + Model                    
DTCWT + Bilateral       0.620134  
DTCWT + Gaussian        0.628490  
DTCWT + Median          0.635333  
DTCWT + Multivariate    0.623838  
DTCWT + Savitzky-Golay  0.617852  


***ANN***

In [7]:
# Filtering functions
def apply_dtcwt_filter(signal):
    coeffs = pywt.wavedec(signal, 'db4', level=1)
    coeffs[1:] = [np.zeros_like(i) for i in coeffs[1:]]
    return pywt.waverec(coeffs, 'db4')[:len(signal)]

def apply_savgol_filter(signal):
    return savgol_filter(signal, window_length=min(len(signal), 21), polyorder=2)

def apply_bilateral_filter(signal):
    return bilateralFilter(signal.astype(np.float32), 9, 75, 75)

def apply_median_filter(signal):
    return medfilt(signal, kernel_size=3)

def apply_multivariate_filter(signal):
    return np.convolve(signal, np.ones(5)/5, mode='same')

def apply_gaussian_filter(signal):
    return gaussian_filter(signal, sigma=1)

def apply_wavelet_denoising(signal):
    return denoise_wavelet(signal, method='BayesShrink', mode='soft', rescale_sigma=True)

# Filtering options
filters = {
    "DTCWT + Savitzky-Golay": lambda x: apply_savgol_filter(apply_dtcwt_filter(x)),
    "DTCWT + Bilateral": lambda x: apply_bilateral_filter(apply_dtcwt_filter(x)),
    "DTCWT + Median": lambda x: apply_median_filter(apply_dtcwt_filter(x)),
    "DTCWT + Multivariate": lambda x: apply_multivariate_filter(apply_dtcwt_filter(x)),
    "DTCWT + Gaussian": lambda x: apply_gaussian_filter(apply_dtcwt_filter(x)),
    "DTCWT + Wavelet Denoising": lambda x: apply_wavelet_denoising(apply_dtcwt_filter(x))
}

# Prepare results
results = []

for filter_name, filter_func in filters.items():
    filtered_data = y.copy()
    for col in targets:
        filtered_data[col] = filter_func(y[col])

    # Standardization
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(filtered_data.loc[y_train.index])
    y_test_scaled = scaler_y.transform(filtered_data.loc[y_test.index])

    # Train ANN model
    for i, target in enumerate(targets):
        ann_model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=100, random_state=42,verbose=True)
        ann_model.fit(X_train_scaled, y_train_scaled[:, i])

        y_pred_ann = ann_model.predict(X_test_scaled)
        rmse_ann = np.sqrt(mean_squared_error(y_test_scaled[:, i], y_pred_ann))
        mae_ann = mean_absolute_error(y_test_scaled[:, i], y_pred_ann)

        results.append([filter_name, target, rmse_ann, mae_ann])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Filter + Model', 'Target Variable', 'RMSE', 'MAE'])

# Print results in a structured way
print(results_df.pivot(index='Filter + Model', columns='Target Variable', values=['RMSE', 'MAE']))

Iteration 1, loss = 0.19355937
Iteration 2, loss = 0.12146921
Iteration 3, loss = 0.10897817
Iteration 4, loss = 0.10382465
Iteration 5, loss = 0.10041296
Iteration 6, loss = 0.09820451
Iteration 7, loss = 0.09666824
Iteration 8, loss = 0.09523918
Iteration 9, loss = 0.09421598
Iteration 10, loss = 0.09310146
Iteration 11, loss = 0.09235230
Iteration 12, loss = 0.09158040
Iteration 13, loss = 0.09087600
Iteration 14, loss = 0.09019016
Iteration 15, loss = 0.08962413
Iteration 16, loss = 0.08918563
Iteration 17, loss = 0.08847256
Iteration 18, loss = 0.08795073
Iteration 19, loss = 0.08759927
Iteration 20, loss = 0.08725159
Iteration 21, loss = 0.08692008
Iteration 22, loss = 0.08649023
Iteration 23, loss = 0.08639132
Iteration 24, loss = 0.08597669
Iteration 25, loss = 0.08592693
Iteration 26, loss = 0.08550930
Iteration 27, loss = 0.08522695
Iteration 28, loss = 0.08508793
Iteration 29, loss = 0.08494898
Iteration 30, loss = 0.08466175
Iteration 31, loss = 0.08449946
Iteration 32, los